In [1]:
import sys 
import csv
import os
from datetime import datetime
from datetime import timedelta
import numpy as np

#Retrieve stock price for a particular day
def retrieveStockPrice(company):
    stock_file = os.path.join("SandP500_stock", company + "_data.csv")
    with open(stock_file) as stockfile:
        stockprice = {}
        reader = csv.DictReader(stockfile)
        for row in reader:
            if 'open' in row:
                if row["open"] and row['close']:
                    stockprice[row['date']] = round((float(row["open"]) + float(row['close'])) / 2, 5)
            else:
                if row["Open"] and row['Close']:
                    stockprice[row['Date']] = round((float(row["Open"]) + float(row['Close'])) / 2, 5)
    return stockprice

#Retriev accumulated sentiment and compare with actual stock price
company_list = []
with open('SP500.csv') as SP500csv:
    reader = csv.DictReader(SP500csv)
    for row in reader:
        company_list.append(row['Symbol'])

def runROI(source_dir, year, month, max_day, user_id):
    day = max_day        
    n_company = len(company_list)
    ROI_history = [0] * n_company
    for i in range(n_company):
        ROI_history[i] = [0] * day

    currHold = 0
    path = os.path.join("Extra_Storage", source_dir, year, month, "User_Sentiment")
    with open(os.path.join( path, 'acc_sentiment_' + year + '_' + month + '_' + user_id + '.csv')) as accSentiment:
        reader = csv.DictReader(accSentiment)
        company_idx = 0
        for acc_Sentiment in reader:
            currHold = 0
            company = company_list[company_idx]
            avg_price = retrieveStockPrice(company)
            if year + '-' + month + '-01' in avg_price:
                previousPrice = avg_price[year + '-' + month + '-01']
            else:
                previousPrice = -999
            sumSentiment = 0
            for day in range(1, max_day):
    #            if day < 31:
    #                date = '4/' + str(day) + '/2016'
    #                month = 4
    #                next_date = datetime(2016, month, day) + timedelta(days=1)  #Buy stock 1 day after
    #            else:
    #                date = '5/' + str(day-30) + '/2016'
    #                month = 5
    #                next_date = datetime(2016, month, day-30) + timedelta(days=1)  #Buy stock 1 day after
                next_date = datetime(int(year), int(month), day) + timedelta(days=1)  #Buy stock 1 day after
                date =  str(int(month)) + '/' + str(day) + '/' + year
                sumSentiment += int(acc_Sentiment[date])

                if next_date.month < 10:
                    next_date_string_month = str(next_date.year) + '-0' + str(next_date.month)
                else:
                    next_date_string_month = str(next_date.year) + '-' + str(next_date.month)
                if next_date.day < 10:
                    next_date_string = str(next_date_string_month) +  '-0' + str(next_date.day)
                else:
                    next_date_string = str(next_date_string_month) +  '-' + str(next_date.day)

                if next_date_string in avg_price:
                    nextdayPrice = avg_price[next_date_string]                
                    if sumSentiment > 0:
                        if currHold == 0:
                            currHold = 100 / nextdayPrice
                            previousPrice = avg_price[next_date_string]
                    elif sumSentiment < 0:
                        if currHold:
                            currHold = 0

                    if currHold:       
                        ROI_history[company_idx][day] = round(ROI_history[company_idx][day-1] + (nextdayPrice-previousPrice)*currHold , 5)
                    else:
                        ROI_history[company_idx][day] = round(ROI_history[company_idx][day-1], 5)

                    previousPrice = nextdayPrice #Save stock price for next iteration
                    sumSentiment = 0 #Reset sentiment
                else:
                    ROI_history[company_idx][day] = round(ROI_history[company_idx][day-1], 5)

            company_idx+=1 #Predict next stock
    return ROI_history

def runAllROI(source_dir, year, month, days):
    path = os.path.join("Extra_Storage", source_dir, year, month)
    with open(os.path.join(path, "id_list.txt" )) as id_list:
        reader = id_list.readlines()
        idList = [x.strip() for x in reader]
        for user_id in idList:
            ROI = runROI(source_dir, year, month, days, user_id)
            arr = np.array(ROI)
            np.savetxt(os.path.join(path, 'User_ROI', 'ROI_history_'+year+'_'+month+'_' + user_id +'.csv'), arr, fmt='%1.5f', delimiter=',')

#if __name__ == "__main__":
#        runAllROI('2016', '03', 31)

In [2]:
#Generate dates

def addDateTag(source_dir, year, month, days):
    dates = []
    day = days
    for thedate in range(1, day+1):
        dates.append(month + '/'+ str(thedate) + '/' + year)
        #dates += '4/' + str(thedate) + '/2016, '
        #dates += '4/30/2016' #Add final day

    roipath = os.path.join("Extra_Storage", source_dir, year, month, "User_ROI")
    for dirPath, dirNames, fileNames in os.walk(roipath):
        for file in fileNames:
            filepath = os.path.join(dirPath, file)
            with open(filepath) as outcsv:
                r = csv.reader(outcsv)
                data = [line for line in r]
            with open(filepath, 'w', newline='') as outcsv:
                writer = csv.writer(outcsv)
                writer.writerow(dates)
                writer.writerows(data)


#addDateTag('2016', '03', 31)

In [6]:
import os
#Record each users' overall ROI
def accAllROI(source_dir, year, month, sum_day):
    path = os.path.join("Extra_Storage", source_dir, year, month)
    with open(os.path.join(path, "id_list.txt" )) as id_list:
        reader = id_list.readlines()
        idList = [x.strip() for x in reader]

    acc_roi_all = []

    for user_id in idList:
        roipath = os.path.join(path, 'User_ROI', 'ROI_history_'+ year +'_' +  month + '_' + user_id +'.csv')
        acc_roi = 0
        with open(roipath) as roicsv:
            r= csv.DictReader(roicsv)
            for single_company_roi in r:
                acc_roi += float(single_company_roi[ month +  '/' + str(int(sum_day)) + '/' + year])
            acc_roi_all.append(acc_roi)
    filepath =  os.path.join(path, "Acc_ROI_all_" + year + "_" + month + ".csv" )       
    with open(filepath, 'w', newline='') as outcsv:
        writer = csv.writer(outcsv)
        #for value in acc_roi_all:
        writer.writerows(map(lambda x: [x], acc_roi_all))
        outcsv.close()

#accAllROI("Archiveteam_Filtered", "2016", "03", "31")        

In [3]:
runAllROI("ArchiveteamTest", '2015', '11', 30)
runAllROI("ArchiveteamTest", '2015', '12', 31)
runAllROI("ArchiveteamTest", '2016', '01', 31)
runAllROI("ArchiveteamTest", '2016', '02', 29)
runAllROI("ArchiveteamTest", '2016', '03', 31)
runAllROI("ArchiveteamTest", '2016', '04', 30)
addDateTag("ArchiveteamTest",'2015', '11', 30)
addDateTag("ArchiveteamTest",'2015', '12', 31)
addDateTag("ArchiveteamTest",'2016', '01', 31)
addDateTag("ArchiveteamTest",'2016', '02', 29)
addDateTag("ArchiveteamTest",'2016', '03', 31)
addDateTag("ArchiveteamTest",'2016', '04', 30)
#accAllROI("ArchiveteamTest", "2015", "11", "30")     
#accAllROI("ArchiveteamTest", "2015", "12", "31")
#accAllROI("ArchiveteamTest", "2016", "01", "31")
#accAllROI("ArchiveteamTest", "2016", "02", "29")
#accAllROI("ArchiveteamTest", "2016", "03", "31")
#accAllROI("ArchiveteamTest", "2016", "04", "30")



KeyboardInterrupt: 